In [12]:
from google.cloud import storage
import os
from dotenv import load_dotenv
import json
import sys
import pandas as pd

from google.auth.exceptions import GoogleAuthError
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud import bigquery
from termcolor import colored



ModuleNotFoundError: No module named 'termcolor'

In [4]:
BUCKET_NAME='podcast-ad-skipper'

In [6]:
def auth_gc():
    """Authenticates and returns a Google Cloud Storage client using service account credentials"""
    try:
        with open('gcp/podcast-ad-skipper-0dd8dd2c5ac1.json') as source:
            info = json.load(source)

        storage_credentials = service_account.Credentials.from_service_account_info(info)

        storage_client = storage.Client(project=os.environ.get('GCP_PROJECT_ID'), credentials=storage_credentials)

        print("Authenticated successfully! ✅")
        return storage_client

    except FileNotFoundError:
        print("Error: The specified credentials file 'gcp/file_name.json' was not found.")
        print(colored("Failed to authenticate with Google Cloud Storage ❌", "red"))
        sys.exit(1)

    except GoogleAuthError as e:
        print(f"Error: Authentication failed with Google Cloud: {e}")
        print(colored("Failed to authenticate with Google Cloud Storage ❌", "red"))
        sys.exit(1)

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        print(colored("Failed to authenticate with Google Cloud Storage ❌", "red"))
        sys.exit(1)

In [7]:
storage_client = auth_gc()

Error: The specified credentials file 'gcp/file_name.json' was not found.


NameError: name 'colored' is not defined

In [10]:
def create_spectrogram(audio_file):
    #y is an array representing the frequency of the audio signal at each sample.
    #sr is the sampling rate (samples per second),
    y, sr = librosa.load(audio_file)
    spectrogram = librosa.stft(y)

    #Transforms to decibel scale (logarithmic), which gives more emphasis to amplitude changes in high volumes
    spectrogram_db = librosa.amplitude_to_db(abs(spectrogram))
    return spectrogram_db

In [11]:
def spectrogram_to_numpy(spectrogram_db):
    return np.array(spectrogram_db)


In [14]:
def process_audio_file(BUCKET_NAME, blob_name):
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(blob_name)

    with blob.open('rb') as blob_file:
        file_data = blob.file.read()

    spectrogram_db = create_spectrogram(file_data)
    spectrogram_to_numpy(file_data)

In [15]:
list_np_arrays = []

def process_all_audio_files_in_bucket(BUCKET_NAME):
    bucket = storage_client.bucket(BUCKET_NAME)
    blob_list = bucket.list_blobs()
    for blob in blob_list:
        np_array = process_audio_file(BUCKET_NAME, blob)
        list_np_arrays.append(np_array)
        

In [16]:
len(list_np_arrays)


0

In [18]:
list_np_arrays[0].shape()

IndexError: list index out of range